---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [371]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [372]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [173]:
def answer_one():
    
    
    return (sum(spam_data['target'])/len(spam_data['target']))*100

In [174]:
#answer_one()

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [175]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vector = CountVectorizer().fit(X_train)
    features = vector.get_feature_names()
    longest = ''
    for word in features:
        if len(word) > len(longest):
            longest = word
    return longest

In [176]:
#answer_two()

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [177]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vector = CountVectorizer().fit(X_train)
    X_train_vectorized = vector.transform(X_train)
    
    md = MultinomialNB(alpha=0.1).fit(X_train_vectorized, y_train)
    
    X_test_vectorized = vector.transform(X_test)
    prediction = md.predict(X_test_vectorized)
    
    return roc_auc_score(y_test, prediction)

In [178]:
#answer_three()

In [179]:
#features[sorted_coefs_indexes][:10]

In [180]:
#features[sorted_coefs_indexes][:-11:-1]

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [181]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_transformed = vect.transform(X_train)
    
    tfidf_scores = pd.DataFrame(X_train_transformed.max(0).T.todense(), index=vect.get_feature_names())
    
    top20 = tfidf_scores[0].sort_values(ascending=False)[:20]
    bottom20 = tfidf_scores[0].sort_values(ascending=False)[:-21:-1]
    
    # reset the index, sort by TFIDF (descending) and index (ascending), and put the index back in place
    top20 = top20.reset_index().sort_values([0, 'index'], ascending=[False, True]).set_index('index')
    # reset the index, sort by TFIDF (ascending) and index (ascending), and put the index back in place
    bottom20 = bottom20.reset_index().sort_values([0, 'index'], ascending=[True, True]).set_index('index')
    
    top20 = pd.Series(top20[0])
    bottom20 = pd.Series(bottom20[0])
    
    # remove the index name and Series name for the autograder requirements
    top20.index.name = None
    top20.name=None
    
    bottom20.index.name = None
    bottom20.name= None
    return (bottom20, top20)

In [301]:
#answer_four()

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [324]:
def answer_five():
    
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_transformed = vect.transform(X_train)

    from sklearn.naive_bayes import MultinomialNB

    md = MultinomialNB(alpha=0.1).fit(X_train_transformed, y_train)

    prediction = md.predict_proba(vect.transform(X_test))[:, 1]

    from sklearn.metrics import roc_auc_score
   
    return roc_auc_score(y_test, prediction)

In [316]:
#answer_five()

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [185]:
def answer_six():
    not_spam_len = spam_data[spam_data['target'] == 0]['text'].agg(len)
    not_spam_avg = not_spam_len.mean()

    spam_len = spam_data[spam_data['target'] == 1]['text'].agg(len)
    spam_avg = spam_len.mean()
    
    return (not_spam_avg, spam_avg)

In [299]:
#answer_six()

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [187]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [188]:
from sklearn.svm import SVC

def answer_seven():
    
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_transformed = vect.transform(X_train)

    added_train = add_feature(X_train_transformed, X_train.agg(len))

    md = SVC(C=10000).fit(added_train, y_train)

    X_test_transformed = vect.transform(X_test)
    added_test = add_feature(X_test_transformed, X_test.agg(len))

    prediction = md.predict(added_test)

    
    return roc_auc_score(y_test, prediction)

In [298]:
#answer_seven()

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [213]:
def answer_eight():
    
    not_spam = spam_data[spam_data['target'] == 0]['text']
    spam = spam_data[spam_data['target'] == 1]['text']
    
    not_spam_avg = not_spam.str.findall(r'\d').agg(len).mean()
    spam_avg = spam.str.findall(r'\d').agg(len).mean()
    
    return(not_spam_avg, spam_avg)

In [297]:
#answer_eight()

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [378]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    # Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than 5 and using word n-grams from n=1 to n=3
    vect= TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    # Transform X_train
    X_train_trans = vect.transform(X_train)
    # Add feature #1: the length of document (number of characters)
    temp1 = add_feature(X_train_trans, X_train.agg(len))
    # Add feature #2: number of digits per document
    X_train_features = add_feature(temp1, X_train.agg(lambda x: x.str.findall(r'\d').agg(len)))
    
    # Train the model
    md = LogisticRegression(C=100).fit(X_train_features, y_train)

    # Transform the Test data using the same methods
    X_test_trans = vect.transform(X_test)
    temp2 = add_feature(X_test_trans, X_test.agg(len))
    X_test_features = add_feature(temp2, X_test.agg(lambda x: x.str.findall(r'\d').agg(len)))

    # predict
    prediction = md.predict(X_test_features)
    
    return roc_auc_score(y_test, prediction)

In [382]:
X_train_features.shape

(4179, 16387)

In [296]:
#answer_nine()

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [244]:
def answer_ten():
    not_spam = spam_data[spam_data['target'] == 0]['text']
    spam = spam_data[spam_data['target'] == 1]['text']

    not_spam_avg = not_spam.str.findall(r'\W').agg(len).mean()
    spam_avg = spam.str.findall(r'\W').agg(len).mean()
    
    return (not_spam_avg, spam_avg)

In [295]:
#answer_ten()

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [373]:
def answer_eleven():
    # Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than 5 and using word n-grams from n=1 to n=3
    vect= CountVectorizer(min_df=5, ngram_range=(1,5), analyzer='char_wb').fit(X_train)
    # Transform X_train
    X_train_trans = vect.transform(X_train)
    # Add feature #1: the length of document (number of characters)
    temp1 = add_feature(X_train_trans, X_train.agg(len))
    # Add feature #2: number of digits per document
    temp2 = add_feature(temp1, X_train.agg(lambda x: x.str.findall(r'\d').agg(len)))
    # Add feature #3: number of non-word characters (anything other than a letter, digit or underscore.)
    X_train_features = add_feature(temp2, X_train.agg(lambda x: x.str.findall(r'\W').agg(len)))

    X_test_trans = vect.transform(X_test)
    # Add feature #1: the length of document (number of characters)
    temp3 = add_feature(X_test_trans, X_test.agg(len))
    # Add feature #2: number of digits per document
    temp4 = add_feature(temp3, X_test.agg(lambda x: x.str.findall(r'\d').agg(len)))
    # Add feature #3: number of non-word characters (anything other than a letter, digit or underscore.)
    X_test_features = add_feature(temp4, X_test.agg(lambda x: x.str.findall(r'\W').agg(len)))

    # Train the model
    md = LogisticRegression(C=100).fit(X_train_features, y_train)

    # predict probabilities. Column 0 is the probability that it's class 0; Column 1 - class 1 (spam)
    prediction = md.predict_proba(X_test_features)[:, 1]
    # auc_score 
    auc_score = roc_auc_score(y_test, prediction)
    
    # Get coefficients
    sorted_coefs = md.coef_[0]

    # get the features + add newly added features
    features = np.array(vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])

    # put things in a dataframe
    feature_df = pd.DataFrame([features, sorted_coefs]).T

    # sort the df by coefficients
    sorted_df = feature_df.sort_values(1, ascending=False)

    # get top and bottom 10 coefficient names
    top10 = sorted_df[:10][0]
    bottom10 = sorted_df[:-11:-1][0]
    
    # remove Series name
    top10.name = None
    bottom10.name = None

    # remove indexes names
    top10.index.name = None
    bottom10.index.name = None
    
    return (auc_score, bottom10, top10)

In [374]:
answer_eleven()

(0.99539497139364719, 3232      .
 3109      )
 8393      h
 1144     go
 4550      ?
 3242     ..
 4528      ;
 0          
 3156      ,
 4498     :)
 dtype: object, 16385            digit_count
 3519                       0
 15938                      x
 10959                     ne
 16386    non_word_char_count
 289                       a 
 659                       ch
 5193                      ar
 8847                      ia
 3464                       /
 dtype: object)

In [375]:
# It is funny that you can use just these 3 features and get roc_auc of 0.98, with no need for any words as features
spam_data['word_count'] = spam_data['text'].agg(len)

spam_data['digits'] = spam_data['text'].agg(lambda x: x.str.findall(r'\d').agg(len))

spam_data['white_space'] = spam_data['text'].agg(lambda x: x.str.findall(r'\W').agg(len))

X_train1, X_test1, y_train1, y_test1 = train_test_split(spam_data[['word_count', 'digits', 'white_space']], 
                                                    spam_data['target'], 
                                                    random_state=0)

from sklearn.linear_model import LogisticRegression

lrr = LogisticRegression(C=100).fit(X_train1, y_train1)

prediction = lrr.predict_proba(X_test1)[:, 1]
roc_auc_score(y_test, prediction)

0.98682792047943224

In [362]:
md.predict_proba([[221, 0, 46]])

array([[ 0.90228864,  0.09771136]])

In [377]:
spam_data.shape

(5572, 5)